# Yelp API Businesses Data Collection Demo
A trial to help construct the data collection pipeline


In [1]:
from yelpapi import YelpAPI
import auth
import http.client
import json
import urllib.parse
from time import sleep
from kafka import KafkaProducer

In [2]:
# Client id and API key definition
client_id = auth.client_id
api_key = auth.api_key

API_HOST = 'api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'
CATEGORIES_PATH = '/v3/categories'

headers = {'Authorization':'Bearer %s' % api_key}

In [3]:
# Set up kafka producer
producer = KafkaProducer(bootstrap_servers=['localhost:9093'],
                         api_version=(0,10,1))

In [4]:
yelp_api = YelpAPI(api_key)

connection = http.client.HTTPSConnection(API_HOST)
connection.request("GET",CATEGORIES_PATH,headers=headers)

connection_response = connection.getresponse()
response_data = connection_response.read()
#categories = json.loads(response_data.decode("utf-8"))

In [5]:
# Extract all categories
categories = json.loads(response_data.decode("utf-8"))
#categories['categories'][0]

## Business Search
Yelp API allows us to search for businesses within a maximum of 40000 meters (25 miles) approximately. NOTE: The actual search radius may be lower than the suggested radius in dense urban areas, and higher in regions of less business density.

In [6]:
mid_point_london_latlng = [51.509835,-0.111440]
radius_m = 40000  # maximum search radius in meters
offset = 0        # initial offset
result_limit = 50 # maximum allowable
total_result_limit = 1000 # api total number of result limit
offset_shift = 50
term = ""#category_list[0]#""
term_cnt = 1

response_list = []
search = True

In [7]:
# BUSINESS SEARCH BY TERM
def GetAPIData(term):
    print("Getting Yelp businesses data for category: ", term, "...")
    mid_point_london_latlng = [51.509835,-0.111440]
    radius_m = 40000  # maximum search radius in meters
    offset = 0        # initial offset
    result_limit = 50 # maximum allowable
    total_result_limit = 1000 # api total number of result limit
    offset_shift = 50
    
    response_list = []
    search = True
    
    cnt=0
    
    while search:
        try:
            response = yelp_api.search_query(
                term=term,
                latitude=mid_point_london_latlng[0],
                longitude=mid_point_london_latlng[1],
                radius=radius_m, offset=offset,
                limit=result_limit)

            for business in response['businesses']:
                cnt+=1
                producer.send('zoo-lion',json.dumps(business).encode('utf-8'))
                print("     Sent business data ",cnt," for id: ", business["id"],"\r", end="")
                sleep(5)
            
            response_list.append(response)
            sleep(2)

            if offset == 0:
                offset = offset_shift + 1

            elif result_limit + offset + offset_shift > total_result_limit and offset <= total_result_limit:
                offset += offset_shift
                result_limit = total_result_limit - offset
                # The allowable limit if 1000 result so i have adjusted the
                # result limit so we take all of the allowable 1000 result

            else:
                offset += offset_shift

        except Exception:
            break
    
    return response_list

In [8]:
GetAPIData("")
producer.send('zoo-lion',b'end')

Getting Yelp businesses data for category:   ...


KeyboardInterrupt: 